In [1]:
import cv2
import glob
import tensorflow as tf
import numpy as np
import pickle
import os

In [2]:
# image related data
images_name = []
images_list = []
images_features = {}
img_features_path = os.path.join('../dataset/pkl/train2048.pkl')

# caption related data
captions_list = []
captions_mapped = {}
vocab = {}

# encode model for preprocessing
encode_model = None

In [3]:
path = '../dataset/images/Flicker8k_Dataset/*.jpg'
for img in glob.glob(path):
    if not os.path.exists(img_features_path):
        image = cv2.imread(img)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (299,299))
        images_list.append(image)
    img = img.split('/')[-1]
    images_name.append(img)

In [4]:
x = tf.keras.applications.InceptionV3(include_top=True, weights='imagenet')
encode_model = tf.keras.models.Model(x.input, x.layers[-2].output)

In [5]:
if not os.path.exists(img_features_path):
    for j in range(0,len(images_list)):
        i = images_list[j].reshape(1,299,299,3)
        i = encode_model.predict(i).reshape(2048,)
        images_features[images_name[j]] = i

    with open(img_features_path, 'wb') as f:
        pickle.dump(self.images_features, f)

else:
    print('File already exist, opening...')
    with open(img_features_path, 'rb') as f:
        images_features = pickle.load(f)

File already exist, opening...


In [6]:
f = open('../dataset/text/Flickr8k.token.txt', 'rb')
captions_list = f.read().decode('utf-8').split('\n')

for i in captions_list:
    name = i.split('\t')[0][:-2]
    caption = i.split('\t')[1]
    caption = caption.lower()
    caption = 'sequencestart ' + caption + ' sequenceend'

    if name in images_name:
        if name not in captions_mapped:
            captions_mapped[name] = [caption]
        else:
            captions_mapped[name].append(caption)

In [7]:
mapped_number = 1
for w in captions_mapped.values():
    for words in w:
        for word in words.split():
            if word not in vocab:
                vocab[word] = mapped_number
                mapped_number += 1

for i, w in captions_mapped.items():
    for words in w:
        mapped = []
        for word in words.split():
            mapped.append(vocab[word])

        captions_mapped[i][w.index(words)] = mapped

In [8]:
len(captions_mapped)

8091

In [9]:
captions_mapped['1000268201_693b08cb0e.jpg']

[[1, 2, 3, 4, 2, 5, 6, 7, 8, 9, 2, 10, 11, 12, 4, 13, 14, 15, 16, 17],
 [1, 2, 18, 19, 20, 2, 21, 22, 16, 17],
 [1, 2, 23, 18, 8, 20, 2, 21, 24, 16, 17],
 [1, 2, 23, 18, 8, 25, 12, 26, 27, 24, 16, 17],
 [1, 2, 23, 18, 4, 2, 5, 6, 19, 20, 2, 21, 28, 16, 17]]

In [10]:
MAX_LEN = 0
for img_name, captions in captions_mapped.items():
    for caption in captions:
        if len(caption) > MAX_LEN:
            MAX_LEN = len(caption)

In [11]:
MAX_LEN

40

In [12]:
def generator():
    X = []
    y_in = []
    y_out = []
    j = 0
    while True:
        for img_name, captions in captions_mapped.items():
            j += 1
            for caption in captions:
                for i in range(1, len(caption)):
                    X.append(images_features[img_name])

                    in_seq = [caption[:i]]
                    out_seq = caption[i]

                    in_seq = tf.keras.preprocessing.sequence.pad_sequences(
                        in_seq, 
                        maxlen=MAX_LEN, 
                        padding='post', 
                        truncating='post')[0]

                    out_seq = tf.keras.utils.to_categorical([out_seq], num_classes=len(vocab) + 1)[0]

                    y_in.append(in_seq)
                    y_out.append(out_seq)
            if j == 1500:
                yield (np.array(X), np.array(y_in), np.array(y_out))
                X = []
                y_in = []
                y_out = []
                j=0